In [383]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
import random

In [503]:
dataset = pd.read_csv("train.csv", index_col="PassengerId")
print("Initial dataset shape:{}".format(dataset.shape))
#print(dataset.head())
def pipe(dataset):
    
    y = dataset["Survived"]
    dataset = dataset.drop(["Survived"], axis=1)
    x = pretify(dataset, False)
    return (x,y)

ohe = preprocessing.OneHotEncoder()

def pretify(dataset, prod=True):
    x = dataset.drop(["Cabin", "Embarked","Sex", "Name", "Ticket"], axis=1)
    x["Age"] = x["Age"].fillna(0)
    x["Fare"] = x["Fare"].fillna(0)
    
    ix=[i for i in dataset.columns if i in ["Embarked", "Sex"]]  #without Cabin and Ticket
    
    x_cat = dataset[ix];
    
    #print(x_cat.shape)
    
    cabin_list = list("ABCDEFTG")
    def map_cabin(cabin):
        #for substring in str(cabin):
            #if substring in cabin_list:
                #return substring
        
        if cabin is not None:
            return str(cabin)
        
        return "U"
    
    #x_cat["Cabin"] = x_cat["Cabin"].map(map_cabin)
    
    towns = list("CQS")
    x_cat["Embarked"] = x_cat["Embarked"].fillna(random.choice(towns))
    
    #print(x_cat.head())
    
    x_cat = x_cat.apply(preprocessing.LabelEncoder().fit_transform)
    
    #print(x_cat.head())
    
    if not prod:
        
        ohe.fit(x_cat)
    
    
    x_cat_encoded = ohe.transform(x_cat)
    
    
    #x = np.column_stack((x, x_cat))
    #print(x_cat_encoded.toarray().shape)
    x = np.column_stack((x, x_cat_encoded.toarray()))
    
    print(x.shape)
    
    return x

dataset_X, dataset_y = pipe(dataset)
print(dataset_X.shape)

Initial dataset shape:(891, 11)
(891, 10)
(891, 10)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [504]:
from sklearn.model_selection import train_test_split
train_X , test_X, train_y, test_y = train_test_split(dataset_X, dataset_y, random_state=26)

print(train_X.shape)


(668, 10)


In [505]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_y)

def print_score(train, test):
    print("Train score is {}".format(train))
    print("Test score is {}".format(test))
    print("")

print_score(model.score(train_X, train_y), model.score(test_X, test_y))
#alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
alphas = [100]
for alpha in alphas:
    
    from sklearn.linear_model import RidgeClassifier
    model2 = RidgeClassifier(alpha)
    model2.fit(train_X, train_y)
    print("using alpha = {}".format(alpha))
    print("Train score is {}".format(model2.score(train_X, train_y)))
    print("Test score is {}".format(model2.score(test_X, test_y)))
    print("")

Train score is 0.7919161676646707
Test score is 0.7982062780269058

using alpha = 100
Train score is 0.8083832335329342
Test score is 0.7847533632286996



In [506]:
def save(prediction):
    df = pd.DataFrame(data={"Survived": prediction}, index=test_data["PassengerId"])
    df.to_csv("with_data_cleaning.csv")

In [507]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(train_X, train_y)

print_score(knn_model.score(train_X, train_y), knn_model.score(test_X, test_y))

Train score is 0.7919161676646707
Test score is 0.672645739910314



In [508]:
from sklearn.linear_model import SGDClassifier

sdg_model = SGDClassifier()
sdg_model.fit(train_X, train_y)

print_score(knn_model.score(train_X, train_y), knn_model.score(test_X, test_y))

Train score is 0.7919161676646707
Test score is 0.672645739910314



/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [509]:
from sklearn.ensemble import RandomForestClassifier

random_f_model = RandomForestClassifier(random_state=0)
random_f_model.fit(train_X, train_y)

print_score(random_f_model.score(train_X, train_y), random_f_model.score(test_X, test_y))

Train score is 0.968562874251497
Test score is 0.820627802690583



In [510]:
test_data = pd.read_csv("test.csv")
test_data_processed = pretify(test_data.drop(["PassengerId"], axis=1))

#prediction = random_f_model.predict(test_data_processed)
#prediction = model2.predict(test_data_processed)
prediction = model.predict(test_data_processed)


save(prediction)

(418, 10)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
